<a href="https://colab.research.google.com/github/Reennee/database-design/blob/main/Data_Normalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data
The data used in this task was obtained uploaded to OpenML by Dustin Carrion. The link to the dataset is here: https://www.openml.org/search?type=data&sort=runs&id=43454&status=active.

## Structure
The data contains the following columns:

- person_age (int): The age of the borrower
- person_income (int): How much the person makes annually in euros.
- person_home_ownership (enum): Either rent, own, mortgage, or
- person_emp_length (float): How long the persn has been employed (in years)
- loan_intent (enum): The intended purpose of the borrrowed funds.
- loan_grade (enum): The grade of the loan. This is better explained [here](https://www.thebalancemoney.com/what-is-loan-grading-5211003).
- loan_amnt (float): The amount borrowed.
- loan_int_rate (float): The loan interest rate.
- loan_status (enum): The present status of the loan where 0 means they have not defaulted and 1 means they have defaulted.
- loanpercentincome Percent income
- cb_person_default_on_file (enum): Either 0 or 1 where 1 means the borrower has defaulted on loans in the past and 0 means they have not.
- cb_preson_cred_hist_length (float): How long the person has been using credit in years.

## Quantity
There are 32,581 rows in the dataset.


# Setup
Installing additional dependencies.

In [ ]:
!pip install liac-arff

Importing necessary libraries

In [ ]:
import pandas as pd
import arff
import sqlite3

# Reading the data

In [ ]:
# Reading the data from ./credit_risk.arff
path_to_file = './credit.arff'
arff_data = arff.load(open(path_to_file, 'r'))

# Setting columns as these were not preset in the data
df = pd.DataFrame(arff_data['data'], columns=[attr[0] for attr in arff_data['attributes']])

# Printing the first few rows of the data
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [ ]:
def create_map(numpy_array):
  ids = range(1, len(numpy_array) + 1)
  map = dict(zip(numpy_array, ids))
  return map

In [ ]:
person_home_ownership_values = df['person_home_ownership'].unique()
person_home_ownership_map = create_map(person_home_ownership_values)
person_home_ownership_map

{'RENT': 1, 'OWN': 2, 'MORTGAGE': 3, 'OTHER': 4}

In [ ]:
loan_intent_values = df['loan_intent'].unique()
loan_intent_map = create_map(loan_intent_values)
loan_intent_map

{'PERSONAL': 1,
 'EDUCATION': 2,
 'MEDICAL': 3,
 'VENTURE': 4,
 'HOMEIMPROVEMENT': 5,
 'DEBTCONSOLIDATION': 6}

In [ ]:
loan_grade_values = df['loan_grade'].unique()
loan_grade_map = create_map(loan_grade_values)
loan_grade_map

{'D': 1, 'B': 2, 'C': 3, 'A': 4, 'E': 5, 'F': 6, 'G': 7}

In [ ]:
loan_status_values = list(map(lambda v: str(v), df['loan_status'].unique()))
loan_status_map = create_map(loan_status_values)

loan_status_map

{'1': 1, '0': 2}

In [ ]:
loan_default_on_file_values = df['cb_person_default_on_file'].unique()
loan_default_on_file_map = create_map(loan_default_on_file_values)

loan_default_on_file_map

{'Y': 1, 'N': 2}

In [ ]:
conn = sqlite3.connect('credit.db')
cur = conn.cursor()

In [ ]:
def populate_supporting_table(entries, table_name):
  for value, id in entries.items():
    cur.execute(f'INSERT INTO {table_name} VALUES(?, ?)', (int(id), value))

In [ ]:
cur.execute('CREATE TABLE loan_statuses(id INTEGER PRIMARY KEY, status TEXT)')
populate_supporting_table(loan_status_map, 'loan_statuses')

cur.execute('CREATE TABLE home_ownership_types(id INTEGER PRIMARY KEY, type TEXT)')
populate_supporting_table(person_home_ownership_map, 'home_ownership_types')

cur.execute('CREATE TABLE loan_intents(id INTEGER PRIMARY KEY, intent TEXT)')
populate_supporting_table(loan_intent_map, 'loan_intents')

cur.execute('CREATE TABLE loan_default_on_file(id INTEGER, value TEXT)')
populate_supporting_table(loan_default_on_file_map, 'loan_default_on_file')

cur.execute('CREATE TABLE loan_grades(id INTEGER PRIMARY KEY, loan_grades TEXT)')
populate_supporting_table(loan_grade_map, 'loan_grades')

In [ ]:
cur.execute("""CREATE TABLE loans(
  id INTEGER PRIMARY KEY,
  person_age INTEGER,
  person_home_ownership_type_id INTEGER,
  person_emp_length REAL,
  loan_intent_id INTEGER,
  loan_grade_id INTEGER,
  loan_amnt REAL,
  loan_int_rate REAL,
  loan_status_id INTEGER,
  loan_percent_income REAL,
  cb_person_default_on_file INTEGER,
  cb_person_credit_hist_length REAL)""")

for index, row in df.iterrows():
  id = index + 1
  person_age = row['person_age']
  person_home_ownership_id = person_home_ownership_map[row['person_home_ownership']]
  person_emp_length = row['person_emp_length']
  loan_intent_id = loan_intent_map[row['loan_intent']]
  loan_grade_id = loan_grade_map[row['loan_grade']]
  loan_amnt = row['loan_amnt']
  loan_int_rate = row['loan_int_rate']
  loan_status_id = loan_status_map[str(row['loan_status'])]
  loan_percent_income = row['loan_percent_income']
  cb_person_default_on_file = row['cb_person_default_on_file']
  cb_person_credit_hist_length = row['cb_person_cred_hist_length']

  cur.execute("INSERT INTO loans VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (
      id,
      person_age,
      person_home_ownership_id,
      person_emp_length,
      loan_intent_id,
      loan_grade_id,
      loan_amnt,
      loan_int_rate,
      loan_status_id,
      loan_percent_income,
      cb_person_default_on_file,
      cb_person_credit_hist_length
      )
  )

In [ ]:
cur.execute('UPDATE loan_statuses SET status="Default" WHERE id=2')
cur.execute('UPDATE loan_statuses SET status="Not Default" WHERE id=1')

In [ ]:
conn.commit()